In [1]:
#import packages
import pandas as pd
import numpy as np
import scipy as sp #https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.pearsonr.html

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#read in county and drop unrelated variables 
df = pd.read_csv('~/Documents/County_v2_diabetes/cleaned.csv')
df=df.drop(columns=['Unnamed: 0'])
df=df.drop(columns=['5-Digit FIPS Code'])
#df=df.drop(columns=['state_county'])
#http://www.countyhealthrankings.org/explore-health-rankings/what-and-why-we-rank/health-factors/health-behaviors/sexual-activity/teen-births

#clean county data state county var to match with click through rate data
df['state_county']=df['state_county'].str.upper()
df['state_county']=df['state_county'].str.replace(" COUNTY", "")

In [3]:
#read in click data from snowflake
res = pd.read_csv('~/Documents/County_v2_diabetes/result.csv')
res_click = pd.read_csv('~/Documents/County_v2_diabetes/result_click.csv')

#merge click denom and nominator data and compute click through rate 
res_all=res.merge(res_click, left_on='COUNTY_STATE', right_on='COUNTY_STATE', how='left' )
res_all['rate']=res_all['NUM_CLICK']/res_all['NUM_ALL']

In [4]:
#merge click through data with county data
df_all=df.merge(res_all, left_on='state_county', right_on='COUNTY_STATE', how='inner')

df_all=df_all.drop(columns=['state_county'])
df_all=df_all.drop(columns=['NUM_ALL'])
df_all=df_all.drop(columns=['NUM_CLICK'])

In [5]:
#compute basic stats for merged data

#availabe data size
#aval=df_all.count()
#aval=aval.drop(index=['COUNTY_STATE'])

#mean
mean_vec=df_all.mean()

#standard deviation
std_vec=df_all.std()

#lower bound of confidence interval 
ci_l=mean_vec-std_vec

#higher bound of confidence interval 
ci_h=mean_vec+std_vec

In [6]:
#correlation between click through rate and each county measure 
correlation=[]
p=[]
aval=[]
for i in range(df_all.shape[1]-2):
    dat=pd.DataFrame([df_all.iloc[:, 72], df_all.iloc[:, i]]).T.dropna()#72 is outcome, and 0:70 are feature 
    correlation.append(round(sp.stats.pearsonr(dat.iloc[:,0], dat.iloc[:,1])[0],2))
    p.append(round(sp.stats.pearsonr(dat.iloc[:,0], dat.iloc[:,1])[1],4))
    aval.append(dat.shape[0])

In [7]:
correlation.append(1)
p.append(1)
aval.append(0)

In [8]:
correlation1=pd.Series(correlation)
p1=pd.Series(p)
aval1=pd.Series(aval)

correlation1.index=df_all.drop(columns=['COUNTY_STATE']).columns
p1.index=df_all.drop(columns=['COUNTY_STATE']).columns
aval1.index=df_all.drop(columns=['COUNTY_STATE']).columns

In [9]:
#put outputs together and select the features that has p val <0.05
out=pd.concat([mean_vec, ci_l, ci_h, correlation1, aval1, p1], axis=1, sort=False) 
out.columns=['mean', 'CI_lower', 'CI_high', 'correlation', 'n', 'p value']
out1=out #out1=out[out['p value']<0.05]
out1=out1.sort_values('correlation',ascending=False)

In [10]:
out1.shape

(72, 6)

In [11]:
out1.to_csv('out0820_diab.csv')